# Install

In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.2 MB/s eta 0:00:00


# Imports

In [ ]:
from tqdm import tqdm

# Organizando o ambiente

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Jun 24 18:45:37 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory


In [ ]:
def mostra_memoria(lista_mem=['cpu','gpu']):
  """
  Esta função exibe informações de memória da CPU e/ou GPU, conforme parâmetros fornecidos.

  Parâmetros:
  -----------
  lista_mem : list, opcional
      Lista com strings 'cpu' e/ou 'gpu'.
      'cpu' - exibe informações de memória da CPU.
      'gpu' - exibe informações de memória da GPU (se disponível).
      O valor padrão é ['cpu'].

  Saída:
  -------
  A função não retorna nada, apenas exibe as informações na tela.

  Exemplo de uso:
  ---------------
  Para exibir informações de memória da CPU:
      mostra_memoria(['cpu'])

  Para exibir informações de memória da CPU e GPU:
      mostra_memoria(['cpu', 'gpu'])

  Autor: Marcus Vinícius Borela de Castro

  """
  if 'cpu' in lista_mem:
    vm = virtual_memory()
    ram={}
    ram['total']=round(vm.total / 1e9,2)
    ram['available']=round(virtual_memory().available / 1e9,2)
    # ram['percent']=round(virtual_memory().percent / 1e9,2)
    ram['used']=round(virtual_memory().used / 1e9,2)
    ram['free']=round(virtual_memory().free / 1e9,2)
    ram['active']=round(virtual_memory().active / 1e9,2)
    ram['inactive']=round(virtual_memory().inactive / 1e9,2)
    ram['buffers']=round(virtual_memory().buffers / 1e9,2)
    ram['cached']=round(virtual_memory().cached/1e9 ,2)
    print(f"Your runtime RAM in gb: \n total {ram['total']}\n available {ram['available']}\n used {ram['used']}\n free {ram['free']}\n cached {ram['cached']}\n buffers {ram['buffers']}")
    print('/nGPU')
    gpu_info = !nvidia-smi
  if 'gpu' in lista_mem:
    gpu_info = '\n'.join(gpu_info)
    if gpu_info.find('failed') >= 0:
      print('Not connected to a GPU')
    else:
      print(gpu_info)


In [ ]:
mostra_memoria()

Your runtime RAM in gb: 
 total 13.61
 available 12.65
 used 0.67
 free 10.44
 cached 2.4
 buffers 0.1
/nGPU
Sat Jun 24 18:45:37 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+---

### Vinculando pasta do google drive para salvar dados

In [ ]:
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
current_dir = os.getcwd()
print("Current directory:", current_dir)

Current directory: /content


## Fixando as seeds

In [ ]:
import random
import torch
import torch.nn.functional as F
import numpy as np
from torch.utils.data import DataLoader

In [ ]:
def inicializa_seed(num_semente:int=123):
  """
  Inicializa as sementes para garantir a reprodutibilidade dos resultados do modelo.
  Essa é uma prática recomendada, já que a geração de números aleatórios pode influenciar os resultados do modelo.
  Além disso, a função também configura as sementes da GPU para garantir a reprodutibilidade quando se utiliza aceleração por GPU.

  Args:
      num_semente (int): número da semente a ser utilizada para inicializar as sementes das bibliotecas.

  References:
      http://nlp.seas.harvard.edu/2018/04/03/attention.html
      https://github.com/CyberZHG/torch-multi-head-attention/blob/master/torch_multi_head_attention/multi_head_attention.py#L15
  """
  # Define as sementes das bibliotecas random, numpy e pytorch
  random.seed(num_semente)
  np.random.seed(num_semente)
  torch.manual_seed(num_semente)

  # Define as sementes da GPU
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

  #torch.cuda.manual_seed(num_semente)
  #Cuda algorithms
  #torch.backends.cudnn.deterministic = True


In [ ]:
num_semente=123
inicializa_seed(num_semente)

## Definindo Hiperparâmetros iniciais

In [ ]:
def inicia_hparam()->dict:
  # Inicialização dos parâmetros
  hparam = {}
  hparam["num_workers_dataloader"] = 0
  hparam["device"] = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
  if torch.cuda.is_available(): print(torch. cuda. get_device_name(hparam["device"]))
  return hparam

In [ ]:
hparam=inicia_hparam()

Tesla T4


## Preparando para debug e display

https://zohaib.me/debugging-in-google-collab-notebook/

In [ ]:
!pip install -Uqq ipdb
import ipdb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 67.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd

In [ ]:
mostra_memoria(('cpu','gpu'))

Your runtime RAM in gb: 
 total 13.61
 available 12.38
 used 0.94
 free 9.45
 cached 3.12
 buffers 0.1
/nGPU
Sat Jun 24 18:48:29 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8     9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+---

In [ ]:
!pip install transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 95.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 71.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.0 MB/s eta 0:00:00


In [ ]:
import transformers

https://zohaib.me/debugging-in-google-collab-notebook/

In [ ]:
!pip install -Uqq ipdb
import ipdb
# %pdb off # desativa debug em exceção
# %pdb on  # ativa debug em exceção
# ipdb.set_trace(context=8)  para execução nesse ponto

In [ ]:
def config_display():
  """
  Esta função configura as opções de display do Pandas.
  """

  # Configurando formato saída Pandas
  # define o número máximo de colunas que serão exibidas
  pd.options.display.max_columns = None

  # define a largura máxima de uma linha
  pd.options.display.width = 1000

  # define o número máximo de linhas que serão exibidas
  pd.options.display.max_rows = 100

  # define o número máximo de caracteres por coluna
  pd.options.display.max_colwidth = 50

  # se deve exibir o número de linhas e colunas de um DataFrame.
  pd.options.display.show_dimensions = True

  # número de dígitos após a vírgula decimal a serem exibidos para floats.
  pd.options.display.precision = 7


In [ ]:
def config_debug():
  """
  Esta função configura as opções de debug do PyTorch e dos pacotes
  transformers e datasets.
  """

  # Define opções de impressão de tensores para o modo científico
  torch.set_printoptions(sci_mode=True)
  """
    Significa que valores muito grandes ou muito pequenos são mostrados em notação científica.
    Por exemplo, em vez de imprimir o número 0.0000012345 como 0.0000012345,
    ele seria impresso como 1.2345e-06. Isso é útil em situações em que os valores dos tensores
    envolvidos nas operações são muito grandes ou pequenos, e a notação científica permite
    uma melhor compreensão dos números envolvidos.
  """

  # Habilita detecção de anomalias no autograd do PyTorch
  torch.autograd.set_detect_anomaly(True)
  """
    Permite identificar operações que podem causar problemas de estabilidade numérica,
    como gradientes explodindo ou desaparecendo. Quando essa opção é ativada,
    o PyTorch verifica se há operações que geram valores NaN ou infinitos nos tensores
    envolvidos no cálculo do gradiente. Se for detectado um valor anômalo, o PyTorch
    interrompe a execução e gera uma exceção, permitindo que o erro seja corrigido
    antes que se torne um problema maior.

    É importante notar que a detecção de anomalias pode ter um impacto significativo
    no desempenho, especialmente em modelos grandes e complexos. Por esse motivo,
    ela deve ser usada com cautela e apenas para depuração.
  """

  # Configura variável de ambiente para habilitar a execução síncrona (bloqueante) das chamadas da API do CUDA.
  os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
  """
    o Python aguarda o término da execução de uma chamada da API do CUDA antes de executar a próxima chamada.
    Isso é útil para depurar erros no código que envolve operações na GPU, pois permite que o erro seja capturado
    no momento em que ocorre, e não depois de uma sequência de operações que pode tornar a origem do erro mais difícil de determinar.
    No entanto, é importante lembrar que esse modo de execução é significativamente mais lento do que a execução assíncrona,
    que é o comportamento padrão do CUDA. Por isso, é recomendado utilizar esse comando apenas em situações de depuração
    e removê-lo após a solução do problema.
  """

  # Define o nível de verbosity do pacote transformers para info
  transformers.utils.logging.set_verbosity_info()


  """
    Define o nível de detalhamento das mensagens de log geradas pela biblioteca Hugging Face Transformers
    para o nível info. Isso significa que a biblioteca irá imprimir mensagens de log informativas sobre
    o andamento da execução, tais como tempo de execução, tamanho de batches, etc.

    Essas informações podem ser úteis para entender o que está acontecendo durante a execução da tarefa
    e auxiliar no processo de debug. É importante notar que, em alguns casos, a quantidade de informações
    geradas pode ser muito grande, o que pode afetar o desempenho do sistema e dificultar a visualização
    das informações relevantes. Por isso, é importante ajustar o nível de detalhamento de acordo com a
    necessidade de cada tarefa.

    Caso queira reduzir a quantidade de mensagens, comentar a linha acima e
      descomentar as duas linhas abaixo, para definir o nível de verbosity como error ou warning

    transformers.utils.logging.set_verbosity_error()
    transformers.utils.logging.set_verbosity_warning()
  """


  # Define o modo verbose do xmode, que é utilizado no debug
  %xmode Verbose

  """
    Comando usado no Jupyter Notebook para controlar o modo de exibição das informações de exceções.
    O modo verbose é um modo detalhado que exibe informações adicionais ao imprimir as exceções.
    Ele inclui as informações de pilha de chamadas completa e valores de variáveis locais e globais
    no momento da exceção. Isso pode ser útil para depurar e encontrar a causa de exceções em seu código.
    Ao usar %xmode Verbose, as informações de exceção serão impressas com mais detalhes e informações adicionais serão incluídas.

    Caso queira desabilitar o modo verbose e utilizar o modo plain,
    comentar a linha acima e descomentar a linha abaixo:
    %xmode Plain
  """

  """
    Dica:
    1.  pdb (Python Debugger)
      Quando ocorre uma exceção em uma parte do código, o programa para a execução e exibe uma mensagem de erro
      com informações sobre a exceção, como a linha do código em que ocorreu o erro e o tipo da exceção.

      Se você estiver depurando o código e quiser examinar o estado das variáveis ​​e executar outras operações
      no momento em que a exceção ocorreu, pode usar o pdb (Python Debugger). Para isso, é preciso colocar o comando %debug
      logo após ocorrer a exceção. Isso fará com que o programa pare na linha em que ocorreu a exceção e abra o pdb,
      permitindo que você explore o estado das variáveis, examine a pilha de chamadas e execute outras operações para depurar o código.


    2. ipdb
      O ipdb é um depurador interativo para o Python que oferece recursos mais avançados do que o pdb,
      incluindo a capacidade de navegar pelo código fonte enquanto depura.

      Você pode começar a depurar seu código inserindo o comando ipdb.set_trace() em qualquer lugar do
      seu código onde deseja pausar a execução e começar a depurar. Quando a execução chegar nessa linha,
      o depurador entrará em ação, permitindo que você examine o estado atual do seu programa e execute
      comandos para investigar o comportamento.

      Durante a depuração, você pode usar comandos:
        next (para executar a próxima linha de código),
        step (para entrar em uma função chamada na próxima linha de código)
        continue (para continuar a execução normalmente até o próximo ponto de interrupção).

      Ao contrário do pdb, o ipdb é um depurador interativo que permite navegar pelo código fonte em que
      está trabalhando enquanto depura, permitindo que você inspecione variáveis, defina pontos de interrupção
      adicionais e até mesmo execute expressões Python no contexto do seu programa.
  """


In [ ]:
config_display()

In [ ]:
config_debug()

Exception reporting mode: Verbose


## Rastro (neptune.ai)

Gerado rastro da execução no Neptune (detalhes no artigo [Rastro-DM: Mineração de Dados com Rastro](https://revista.tcu.gov.br/ojs/index.php/RTCU/article/view/1664))


### Importação de libraries para Rastro

In [ ]:
!pip install neptune-client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.4/450.4 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.9/135.9 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 95.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 8.1 MB/s eta 0:00:00


In [ ]:
import neptune

/usr/local/lib/python3.10/dist-packages/neptune/internal/backends/hosted_client.py:50: NeptuneDeprecationWarning: The 'neptune-client' package has been deprecated and will be removed in the future. Install the 'neptune' package instead. For more, see https://docs.neptune.ai/setup/upgrading/
  from neptune.version import version as neptune_client_version


In [ ]:
!pip install torchviz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for torchviz: filename=torchviz-0.0.2-py3-none-any.whl size=4131 sha256=502551d9cbc29d8205d1d5de0697d6aa22c960875fe7a3ce8d56af0fc9fbf49c
  Stored in directory: /root/.cache/pip/wheels/4c/97/88/a02973217949e0db0c9f4346d154085f4725f99c4f15a87094
Successfully built torchviz


In [ ]:
from torchviz import make_dot

In [ ]:
import getpass
import copy
import time
import re
import tempfile

### Código Rastro

Busca implementar o rastro proposto em [Rastro-DM: Mineração de Dados com Rastro](https://revista.tcu.gov.br/ojs/index.php/RTCU/article/view/1664), autores Marcus Vinícius Borela de Castro e Remis Balaniuk, com o apoio da [solução Neptune](https://app.neptune.ai/)



In [ ]:
def converte_optimizer_state_dict(parm_optimizer)-> dict:
  """
    Recebe um objeto "parm_optimizer" que é do tipo "torch.optim.Optimizer" e retorna um dicionário
    com informações sobre o otimizador.

    O dicionário de retorno é gerado a partir do estado do otimizador que é extraído da propriedade
    "state_dict()" do objeto "parm_optimizer", seu primeiro grupo de parâmetros do otimizador.
  """
  # return str(hparam['optimizer'])
  return parm_optimizer.state_dict()['param_groups'][0]

In [ ]:
def gera_tag_rastro_experiencia_treino(parm_aula: str, hparam: dict) -> str:
    """
    Gera uma string formatada com informações de hiperparâmetros para ser usada como tag de rastro de experiência de treino.

    Args:
        parm_aula (str): Nome da aula que está sendo treinada.
        hparam (dict): Dicionário contendo os hiperparâmetros utilizados no treinamento.

    Returns:
        str: String formatada com as informações de hiperparâmetros.

    Uso:

    hparam['lista_tag_rastro_experiencia_treino'] =        gera_tag_rastro_experiencia_treino(parm_aula='aula7', hparam=hparam)
    """
    # Inicializa uma lista vazia para armazenar as tags
    lista_tag = []

    # Lista com as chaves dos hiperparâmetros que serão utilizados
    lista_chaves = ['embed_dim', 'leiaute_input', 'dim_feedforward', 'max_seq_length', 'ind_activation_function', 'batch_size', 'learning_rate', 'weight_decay', 'amsgrad', 'decrease_factor_lr', 'max_examples', 'eval_every_steps']

    # Itera pelas chaves da lista e cria uma string com a chave e o valor correspondente em hparam,
    # adicionando essa string à lista_tag
    for chave in lista_chaves:
        if chave in hparam:
          tag = f"{chave} {hparam[chave]}"
          lista_tag.append(tag)

    # Concatena a lista de tags em uma única string, separando cada tag por '|',
    # e adicionando o nome da aula como prefixo
    tag_formatada = f"{parm_aula}|" + "|".join(lista_tag)

    return tag_formatada




In [ ]:
class NeptuneRastroRun():
    """
      Classe para geração de rastro de experimento utilizando a ferramenta Neptune.

      Busca implementar o rastro proposto em [Rastro-DM: Mineração de Dados com Rastro](https://revista.tcu.gov.br/ojs/index.php/RTCU/article/view/1664),
      autores Marcus Vinícius Borela de Castro e Remis Balaniuk, com o apoio da [solução Neptune](https://app.neptune.ai/)

      Attributes:
      -----------
      se_geracao_rastro : bool
          Indica se deve ser gerado rastro de experimento.
      neptune_project : str
          Nome do projeto criado no Neptune.
      tag_contexto_rastro : str
          Nome da tag utilizada para identificar o experimento.
      neptune_api_token : str
          Token utilizado para autenticação na API do Neptune.
      run_neptune : object
          Objeto que representa o experimento no Neptune.
      device : str
          Dispositivo utilizado para o treinamento do modelo.
      tmpDir : str
        Diretório temporário utilizado para salvar gráfico do modelo.
    """
    se_geracao_rastro = True
    neptune_project = ""
    tag_contexto_rastro = ""
    neptune_api_token = ""

    def __init__(self, parm_params:dict,  parm_lista_tag:list = None):
      """
        Método construtor da classe NeptuneRastroRun.

        Args:
        - parm_params: dicionário contendo os parâmetros do modelo.
        - parm_lista_tag: lista contendo tags adicionais para o experimento.
      """
      # print(f"NeptuneRastroRun.init: se_geracao_rastro {self.__class__.se_geracao_rastro} parm_params `{parm_params} ")
      if self.__class__.se_geracao_rastro:
        self.run_neptune = neptune.init_run(project=self.__class__.neptune_project, api_token=self.__class__.neptune_api_token, capture_hardware_metrics=True)
        self.run_neptune['sys/name'] = self.__class__.tag_contexto_rastro
        vparams = copy.deepcopy(parm_params)
        if "optimizer" in vparams:
          vparams["optimizer"] = converte_optimizer_state_dict(vparams["optimizer"])
        if 'criterion'  in vparams:
          vparams["criterion"] = str(vparams["criterion"])
        if 'scheduler'  in vparams:
          vparams["scheduler"] = str(type(vparams["scheduler"]))
        if 'device' in vparams:
          vparams['device'] = str(vparams["device"])
        self.device = vparams["device"]
        for tag in parm_lista_tag:
          self.run_neptune['sys/tags'].add(tag)
        self.run_neptune['parameters'] = vparams
        # self.tmpDir = tempfile.mkdtemp()

    @property
    def run():
      """
      Retorna a instância do objeto run_neptune.
      """
      return self.run_neptune

    @classmethod
    def ativa_geracao_rastro(cls):
      """
      Ativa a geração de rastro.
      """
      cls.se_geracao_rastro = True

    @classmethod
    def def_contexto(cls):
      """
      Define o contexto para a geração de rastro.
      """
      cls.se_geracao_rastro = True

    @classmethod
    def desativa_geracao_rastro(cls):
      """
      Desativa a geração de rastro.
      """
      cls.se_geracao_rastro = False

    @classmethod
    def retorna_status_geracao_rastro(cls):
      """
        Retorna o status da geração de rastro.

        Returns:
        - True se a geração de rastro está ativada, False caso contrário.
      """
      return cls.se_geracao_rastro

    @classmethod
    def retorna_tag_contexto_rastro(cls):
      """
        Retorna a tag do contexto de rastro.
      """
      return cls.tag_contexto_rastro

    @classmethod
    def inicia_contexto(cls, neptune_project, tag_contexto_rastro, neptune_api_token):
      """
      Inicia o contexto de execução no Neptune.

      Args:
          neptune_project (str): Nome do projeto no Neptune.
          tag_contexto_rastro (str): Tag que identifica o contexto de execução no Neptune.
          neptune_api_token (str): Token de acesso à API do Neptune.

      Raises:
          AssertionError: Caso a tag_contexto_rastro possua um ponto (.),
            o que pode gerar erros na gravação de arquivo.
      """
      assert '.' not in tag_contexto_rastro, "NeptuneRastroRun.init(): tag_contexto_rastro não pode possuir ponto, pois será usado para gravar nome de arquivo"
      cls.neptune_api_token = neptune_api_token
      cls.tag_contexto_rastro = tag_contexto_rastro
      cls.neptune_project = neptune_project

    def salva_metrica(self, parm_metricas={}):
      """
        Salva as métricas no Neptune Run caso a geração de rastro esteja ativa.

        Parameters
        ----------
        parm_metricas: dict
            Dicionário contendo as métricas a serem salvas. As chaves devem ser os nomes das métricas e os valores devem ser
            os valores das métricas.
      """
      #print(f"NeptuneRastroRun.salva_metrica: se_geracao_rastro {self.__class__.se_geracao_rastro} parm_metricas:{parm_metricas} ")
      if self.__class__.se_geracao_rastro:
        for metrica, valor in parm_metricas.items():
          self.run_neptune[metrica].append(valor)

    def gera_grafico_modelo(self, loader_train, model):
      """
        Gera um gráfico do modelo e o envia para o Neptune.
        Para gerar o gráfico, um forward pass é realizado em um batch de exemplos
        de treino e o resultado é renderizado como um gráfico de nós conectados.
        O gráfico é salvo em um arquivo .png e enviado para o Neptune como um arquivo anexo.

        Args:
            loader_train (torch.utils.data.DataLoader): DataLoader do conjunto de treinamento.
            model (torch.nn.Module): Modelo a ser visualizado.

        Pendente:
          Evolui para usar from io import StringIO (buffer = io.StringIO()) ao invés de tempdir
      """
      return

      """
      falta ajustar make_dot
      if self.__class__.se_geracao_rastro:
        # efetuar um forward
        batch = next(iter(loader_train))
        # falta generalizar linha abaixo. Criar função que recebe modelo e batch como parâmetro?
        outputs = model(input_ids=batch['input_ids'].to(hparam['device']), attention_mask=batch['attention_mask'].to(hparam['device']), token_type_ids=batch['token_type_ids'].to(hparam['device']), labels=batch['labels'].to(hparam['device']))
        nome_arquivo = os.path.join(self.tmpDir, "modelo "+ self.__class__.tag_contexto_rastro + time.strftime("%Y-%b-%d %H:%M:%S"))
        make_dot(outputs, params=dict(model.named_parameters()), show_attrs=True, show_saved=True).render(nome_arquivo, format="png")
        self.run_neptune["parameters/model_graph"].upload(nome_arquivo+'.png')
        self.run_neptune['parameters/model'] = re.sub('<bound method Module.state_dict of ', '',str(model.state_dict))
      """


    def stop(self):
      """
        Para a execução do objeto Neptune. Todos os experimentos do Neptune são sincronizados com o servidor, e nenhum outro
        experimento poderá ser adicionado a este objeto após a chamada a este método.
      """
      if self.__class__.se_geracao_rastro:
        self.run_neptune.stop()


### Definindo parâmetros para o rastro



In [ ]:
os.environ['NEPTUNE_ALLOW_SELF_SIGNED_CERTIFICATE'] = 'TRUE'
os.environ['NEPTUNE_PROJECT'] = 'marcusborela/IA386DD'
tag_contexto_rastro = 'indir'
os.environ['NEPTUNE_API_TOKEN'] = getpass.getpass('Informe NEPTUNE_API_TOKEN')


Informe NEPTUNE_API_TOKEN··········


In [ ]:
NeptuneRastroRun.inicia_contexto(os.environ['NEPTUNE_PROJECT'], tag_contexto_rastro,  os.environ['NEPTUNE_API_TOKEN'])
#NeptuneRastroRun.desativa_geracao_rastro()

# Data load

In [ ]:
PATH_LOCAL_DATA = '/content/drive/MyDrive/treinamento/202301_IA368DD/indir'

In [ ]:
# path_data = '/content/drive/MyDrive/treinamento/202301_IA368DD/indir/data/train_data_juris_tcu_index_bm25.csv'

PATH_TRAIN_DATA_ZIP = f"{PATH_LOCAL_DATA}/data/train_data_juris_tcu_index.zip"
PATH_TRAIN_DATA = f"{PATH_LOCAL_DATA}/data/train_data_juris_tcu_index.csv"

In [ ]:
os.path.exists(PATH_TRAIN_DATA)

True

In [ ]:
%%time
if not os.path.exists(PATH_TRAIN_DATA):
  import zipfile
  !wget https://github.com/marcusborela/ind-ir/raw/main/data/train_juris_tcu_index/train_data_juris_tcu_index.csv -O {PATH_TRAIN_DATA}

  # Extrair o arquivo zip
  with zipfile.ZipFile(PATH_TRAIN_DATA_ZIP, 'r') as zip_ref:
      zip_ref.extractall(PATH_LOCAL_DATA)

  # Listar os arquivos extraídos
  arquivos_extraidos = zip_ref.namelist()
  # Exibir os arquivos extraídos
  for arquivo in arquivos_extraidos:
      print(arquivo)
  print("File loaded")
else:
  print("File already there!")

File already there!
CPU times: user 866 µs, sys: 0 ns, total: 866 µs
Wall time: 1.2 ms


In [ ]:
df_data = pd.read_csv(PATH_TRAIN_DATA)

In [ ]:
df_data.shape

(282636, 6)

Verificando correção do arquivo!

In [ ]:
print(df_data.isnull().sum())

QUERY_ID          0
DOC_ID            0
RELEVANCE         0
TYPE_RELEVANCE    0
DOC_TEXT          0
QUERY_TEXT        0
Length: 6, dtype: int64


In [ ]:
df_data[['QUERY_TEXT','DOC_TEXT']].applymap(len).describe()

,QUERY_TEXT,DOC_TEXT
count,282636.0000000,282636.0000000
mean,314.4387445,971.9820688
std,159.8934908,671.0375228
min,41.0000000,86.0000000
25%,211.0000000,595.0000000
50%,286.0000000,812.0000000
75%,383.0000000,1162.0000000
max,4212.0000000,3739.0000000


Para cada positivo, tem 2 negativos

In [ ]:
df_data['RELEVANCE'].describe()

count    282636.0000000
mean          0.3333333
std           0.4714054
min           0.0000000
25%           0.0000000
50%           0.0000000
75%           1.0000000
max           1.0000000
Name: RELEVANCE, Length: 8, dtype: float64

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df_data[['QUERY_TEXT','DOC_TEXT']]

,QUERY_TEXT,DOC_TEXT
0,SÚMULA TCU 1: Não se compreendem como vencimen...,"O termo é ""Pessoal"".\nPessoal tem nota de esco..."
1,SÚMULA TCU 1: Não se compreendem como vencimen...,"O termo é ""Competência do TCU"".\nCompetência d..."
2,SÚMULA TCU 1: Não se compreendem como vencimen...,"O termo é ""Responsabilidade"".\nResponsabilidad..."
3,SÚMULA TCU 1: Não se compreendem como vencimen...,"O termo é ""Pensão especial"".\nPensão especial ..."
4,SÚMULA TCU 1: Não se compreendem como vencimen...,"O termo é ""Concessão (Pessoal)"".\nConcessão (P..."
...,...,...
282631,O dever de observância à hierarquia militar nã...,"O termo é ""Responsabilidade subjetiva"".\nRespo..."
282632,O dever de observância à hierarquia militar nã...,"O termo é ""Verba ilegal"".\nVerba ilegal tem no..."
282633,O dever de observância à hierarquia militar nã...,"O termo é ""Manifesta ilegalidade"".\nManifesta ..."
282634,O dever de observância à hierarquia militar nã...,"O termo é ""Dever de lealdade"".\nDever de leald..."


In [ ]:
df_data[['QUERY_TEXT','DOC_TEXT']].values[:3]

array([['SÚMULA TCU 1: Não se compreendem como vencimento, para efeito de concessão da pensão especial com fundamento na Lei nº 3.738, de 04/04/60, as vantagens previstas no art. 184 da Lei nº 1.711, de 28/10/52.',
        'O termo é "Pessoal".\nPessoal tem nota de escopo: "Designação genérica de todos os servidores ou funcionários civis pertencentes ao quadro de pessoal de um órgão ou entidade.".\nPessoal tem nota de escopo: "Tema agrupador para área de atuação do Controle Externo.".\nPessoal é uma generalização de: "Servidor público", "Funcionário público", "Pessoal civil", "Pessoal militar", "Pessoal temporário" e "Colaborador".\nPessoal tem termo relacionado: "Sisac", "Despesa com pessoal" e "HCAAF".\nPessoal tem tradução em espanhol: "Recursos humanos".\nPessoal tem tradução em inglês: "Personal" e "Human resources".'],
       ['SÚMULA TCU 1: Não se compreendem como vencimento, para efeito de concessão da pensão especial com fundamento na Lei nº 3.738, de 04/04/60, as vantagens pr

# Separating train and valid


In [ ]:
hparam['percent_test_size'] = .01 #@param [.10, .15, .20] {type:'raw'}

In [ ]:
X_train, X_valid, Y_train, Y_valid = train_test_split(df_data[['QUERY_TEXT','DOC_TEXT']].values,
                                                      df_data['RELEVANCE'].values,
                                                      test_size=hparam['percent_test_size'],
                                                      stratify=df_data['RELEVANCE'].values, random_state=num_semente)


In [ ]:
X_train.shape

(279809, 2)

In [ ]:
X_train.shape, type(X_valid), type(Y_valid), X_valid.shape, Y_valid.shape

((279809, 2), numpy.ndarray, numpy.ndarray, (2827, 2), (2827,))

In [ ]:
print(np.unique(Y_train, return_counts=True), '\n', np.unique(Y_valid, return_counts=True))

(array([0, 1]), array([186539,  93270])) 
 (array([0, 1]), array([1885,  942]))


In [ ]:
X_train[:2]

array([['A caracterização de jogo de planilha prescinde da intenção de conferir vantagem indevida por parte dos agentes administrativos ou dos prepostos da pessoa jurídica contratada.',
        'O termo é "Jogo de cronograma".\nJogo de cronograma tem definição: "Distorção originada em orçamentos com preços unitários superiores aos de mercado nos serviços a serem executados inicialmente, compensados por reduções significativas nos preços dos serviços a executar no final do contrato, de modo que o valor global do contrato se situe dentro dos valores de mercado. Propicia ao contratado auferir ganhos financeiros às custas da Administração, ou até mesmo paralisar a obra após ter executado os serviços que lhe beneficiam, sob a alegação de que os serviços restantes encontram-se em desequilíbrio econômico-financeiro.".\nJogo de cronograma tem termo relacionado: "Equilíbrio econômico-financeiro".'],
       ['Em regra, nos casos de tomada de contas especial instaurada por inexecução parcial do o

In [ ]:
Y_train[:2]

array([0, 0])

# Tokenizer and Dataset

#### Carga

In [ ]:
hparam['model_name'] = 'unicamp-dl/mMiniLM-L6-v2-pt-v2'

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(hparam['model_name'])

loading file sentencepiece.bpe.model from cache at /root/.cache/huggingface/hub/models--unicamp-dl--mMiniLM-L6-v2-pt-v2/snapshots/a9c0d68bae51b053e85747605350bb781d945712/sentencepiece.bpe.model
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--unicamp-dl--mMiniLM-L6-v2-pt-v2/snapshots/a9c0d68bae51b053e85747605350bb781d945712/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--unicamp-dl--mMiniLM-L6-v2-pt-v2/snapshots/a9c0d68bae51b053e85747605350bb781d945712/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--unicamp-dl--mMiniLM-L6-v2-pt-v2/snapshots/a9c0d68bae51b053e85747605350bb781d945712/tokenizer_config.json


#### Experimentações e computando limites

In [ ]:
tokenizer.pad_token_id,tokenizer.cls_token_id,tokenizer.sep_token_id

(1, 0, 2)

In [ ]:
x='x, it is just a test!'
y='y, it is just a continuation!'

In [ ]:
tokenizer(x,y)['input_ids']

[0,
 1022,
 4,
 442,
 83,
 1660,
 10,
 3034,
 38,
 2,
 2,
 113,
 4,
 442,
 83,
 1660,
 10,
 9454,
 1363,
 38,
 2]

In [ ]:
tokenizer.encode_plus(x,y)['input_ids']

[0,
 1022,
 4,
 442,
 83,
 1660,
 10,
 3034,
 38,
 2,
 2,
 113,
 4,
 442,
 83,
 1660,
 10,
 9454,
 1363,
 38,
 2]

In [ ]:
# df_tokenizado = df_data[['QUERY_TEXT','DOC_TEXT']].applymap(lambda x: tokenizer(x)['input_ids'])

In [ ]:
#df_tokenizado.head()
#print('max:', '\n', df_tokenizado.applymap(len).max(), '\n','mean:', '\n', df_tokenizado.applymap(len).mean(), '\n', 'std:', '\n', df_tokenizado.applymap(len).std())
# del df_tokenizado

### Definindo parâmetros e informações para o rastro (hparam) - sobre dados

In [ ]:
# visto que "max_position_embeddings": 512 em https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/blob/main/config.json

hparam['vocab_size'] = tokenizer.vocab_size
hparam['batch_size'] = 32
hparam['num_epochs'] = 10  #@param {type:"slider", min:1, max:100, step:1}
hparam['max_seq_length'] = 512  #tokenizer.max_model_input_sizes
hparam['num_sentenca_train'] = X_train.shape[0] # ou 37400
hparam['num_sentenca_valid'] = X_valid.shape[0] #


Definindo o prefixo de nome do arquivo

In [ ]:
PATH_TRAIN_LOCAL = f"{PATH_LOCAL_DATA}/train/minilm"

In [ ]:
if not os.path.exists(PATH_TRAIN_LOCAL):
  os.makedirs(PATH_TRAIN_LOCAL)
  print(f'{PATH_TRAIN_LOCAL} pasta criada!')
else:
  print(f'{PATH_TRAIN_LOCAL} pasta já existia!')


/content/drive/MyDrive/treinamento/202301_IA368DD/indir/train/minilm pasta já existia!


## Criando código para o Dataset

In [ ]:
class MyDataset():
    """
      Classe para representar um dataset de texto e classes.
    """
    def __init__(self, texts: np.ndarray, classes:list[int], tokenizer, max_seq_length: int):
      """
      Inicializa um novo objeto MyDataset.

      Args:
          texts (np.ndarray): um array com as strings de texto. Cada linha deve ter 2 strings.
          classes (np.ndarray): um array com as classes de cada texto.
          tokenizer: um objeto tokenizer do Hugging Face Transformers.
          max_seq_length (int): o tamanho máximo da sequência a ser considerado.
      Raises:
          AssertionError: se os parâmetros não estiverem no formato esperado.
      """
      # Verifica se os parâmetros são do tipo esperado
      assert isinstance(texts, np.ndarray), f"Parâmetro texts deve ser do tipo np.ndarray e não {type(texts)}"
      assert texts.shape[1] == 2, "Array must have 2 columns"
      for row in texts:
          assert isinstance(row, np.ndarray) and row.shape == (2,), f"Each row in texts must have 2 elements"
          assert isinstance(row[0], str) and isinstance(row[1], str), f"Each element in texts.row must be a string e não {type(row[0])}"
      assert isinstance(classes,np.ndarray), f'classes deve ser do tipo np.ndarray e não {type(classes)}'
      assert isinstance(classes[0],np.int64), f'classes[0] deve ser do tipo numpy.int64 e não {type(classes[0])} '
      assert isinstance(max_seq_length, int), f'max_seq_length deve ser do tipo int e não {type(max_seq_length)}'
      assert max_seq_length > 100, f'max_seq_length deve ser maior do que 100'

      self.texts = texts
      self.classes = classes
      self.tokenizer = tokenizer
      self.max_seq_length = max_seq_length

      # Salvar os dados dos tensores
      x_data_input_ids = []
      x_data_token_type_ids = []
      x_data_attention_masks = []
      for text_pair in tqdm(texts, desc='encoding text pair'):
          encoding = tokenizer.encode_plus(
              text_pair[0],
              text_pair[1],
              add_special_tokens=True,
              max_length=self.max_seq_length,
              padding='max_length',
              return_tensors = 'pt',
              truncation=True,
              return_attention_mask=True,
              return_token_type_ids=True
          )
          x_data_input_ids.append(encoding['input_ids'].long())
          x_data_token_type_ids.append(encoding['token_type_ids'].long())
          x_data_attention_masks.append(encoding['attention_mask'].long())
      print(F'\tVou converter lista para tensor;  Momento: {time.strftime("[%Y-%b-%d %H:%M:%S]")}')
      # squeeze: vai transformar um tensor de shape [2, 1, 322] em um tensor de shape [2, 322].

      self.x_tensor_input_ids = torch.stack(x_data_input_ids).squeeze(1)
      self.x_tensor_attention_masks = torch.stack(x_data_attention_masks).squeeze(1)
      self.x_tensor_token_type_ids = torch.stack(x_data_token_type_ids).squeeze(1)

    def __len__(self):
        """
          Retorna o tamanho do dataset (= tamanho do array texts)
        """
        return len(self.texts)

    def __getitem__(self, idx):
        """
          Retorna um dicionário com os dados do texto e sua classe correspondente, em um formato que pode
          ser usado pelo dataloader do PyTorch para alimentar um modelo de aprendizado de máquina.
        """
        return {
            'input_ids': self.x_tensor_input_ids[idx],
            'attention_mask': self.x_tensor_attention_masks[idx],
            'token_type_ids': self.x_tensor_token_type_ids[idx],
            # 'labels': int(self.classes[idx])
            # 'labels': torch.tensor(self.classes[idx], dtype=torch.float)
            'labels' : torch.tensor(-1. if self.classes[idx] == 0 else 1., dtype=torch.float)
        }

#### Testando o MyDataset e o Dataloader

In [ ]:
# Cria dados fictícios
texts = np.array([['This is the first text', 'This is the second text'],
                  ['This is text 2.1', 'This is text 2.2'],
                  ['This is text 3.1', 'This is text 3.2'],
                  ['This is text 4.1', 'This is text 4.2'],
                  ['This is text 5.1', 'This is text 5.2'],
                  ['This is text 6.1', 'This is text 6.2'],
                  ['This is text 7.1', 'This is text 7.2']])
classes = np.array([1, 0, 1, 0, 1, 0, 1])


In [ ]:
# Cria um objeto da classe MyDataset
dummy_dataset = MyDataset(texts=texts, classes=classes, tokenizer=tokenizer, max_seq_length=hparam['max_seq_length'])

encoding text pair: 100%|██████████| 7/7 [00:00<00:00, 183.79it/s]

	Vou converter lista para tensor;  Momento: [2023-Jun-24 18:49:52]


In [ ]:
# Testa o método __len__()
assert len(dummy_dataset) == 7

# Testa o método __getitem__()
sample = dummy_dataset[0]
assert set(sample.keys()) == {'input_ids', 'attention_mask', 'token_type_ids', 'labels'} #
assert isinstance(sample['input_ids'], torch.Tensor)
assert sample['input_ids'].shape[0] == hparam['max_seq_length']
assert isinstance(sample['attention_mask'], torch.Tensor)
assert sample['attention_mask'].shape[0] == hparam['max_seq_length']
# assert isinstance(sample['labels'], int)
assert isinstance(sample['labels'], torch.Tensor)
# print(sample['labels'], sample['labels'].shape)


In [ ]:
print(sample)

{'input_ids': tensor([    0,  3293,    83,    70,  5117,  7986,     2,     2,  3293,    83,
           70, 17932,  7986,     2,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1, 

In [ ]:
dummy_loader = DataLoader(dummy_dataset, batch_size=2, shuffle=False, num_workers=hparam['num_workers_dataloader'] )

In [ ]:
mostra_memoria(['cpu','gpu'])

Your runtime RAM in gb: 
 total 13.61
 available 11.73
 used 1.56
 free 7.02
 cached 4.92
 buffers 0.12
/nGPU
Sat Jun 24 18:49:52 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8     9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+--

In [ ]:
from transformers import BatchEncoding

In [ ]:
first_batch = next(iter(dummy_loader))

In [ ]:
BatchEncoding(first_batch).to(hparam['device'])

{'input_ids': tensor([[   0, 3293,   83,  ...,    1,    1,    1],
        [   0, 3293,   83,  ...,    1,    1,    1]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], device='cuda:0'), 'labels': tensor([1.0000e+00, -1.0000e+00], device='cuda:0')}

In [ ]:
first_batch

{'input_ids': tensor([[   0, 3293,   83,  ...,    1,    1,    1],
         [   0, 3293,   83,  ...,    1,    1,    1]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]),
 'labels': tensor([1.0000e+00, -1.0000e+00])}

Confirmando efeito do parâmetro shuffle em DataLoader

In [ ]:
dummy_loader = DataLoader(dummy_dataset, batch_size=2, shuffle=False, num_workers=hparam['num_workers_dataloader'])
for ndx, epoch in tqdm(enumerate(range(hparam['num_epochs'])), desc='Epochs'):
  for batch in dummy_loader:
    print(batch['labels'])
  print(f"Fim época {ndx}")


Epochs: 10it [00:00, 188.16it/s]

tensor([1.0000e+00, -1.0000e+00])
tensor([1.0000e+00, -1.0000e+00])
tensor([1.0000e+00, -1.0000e+00])
tensor([1.0000e+00])
Fim época 0
tensor([1.0000e+00, -1.0000e+00])
tensor([1.0000e+00, -1.0000e+00])
tensor([1.0000e+00, -1.0000e+00])
tensor([1.0000e+00])
Fim época 1
tensor([1.0000e+00, -1.0000e+00])
tensor([1.0000e+00, -1.0000e+00])
tensor([1.0000e+00, -1.0000e+00])
tensor([1.0000e+00])
Fim época 2
tensor([1.0000e+00, -1.0000e+00])
tensor([1.0000e+00, -1.0000e+00])
tensor([1.0000e+00, -1.0000e+00])
tensor([1.0000e+00])
Fim época 3
tensor([1.0000e+00, -1.0000e+00])
tensor([1.0000e+00, -1.0000e+00])
tensor([1.0000e+00, -1.0000e+00])
tensor([1.0000e+00])
Fim época 4
tensor([1.0000e+00, -1.0000e+00])
tensor([1.0000e+00, -1.0000e+00])
tensor([1.0000e+00, -1.0000e+00])
tensor([1.0000e+00])
Fim época 5
tensor([1.0000e+00, -1.0000e+00])
tensor([1.0000e+00, -1.0000e+00])
tensor([1.0000e+00, -1.0000e+00])
tensor([1.0000e+00])
Fim época 6
tensor([1.0000e+00, -1.0000e+00])
tensor([1.0000e+00, -

In [ ]:
dummy_loader = DataLoader(dummy_dataset, batch_size=2, shuffle=True, num_workers=hparam['num_workers_dataloader'])
for ndx, epoch in tqdm(enumerate(range(hparam['num_epochs'])), desc='Epochs'):
  for batch in dummy_loader:
    print(batch['labels'])
  print(f"Fim época {ndx}")

Epochs: 10it [00:00, 195.58it/s]

tensor([-1.0000e+00, 1.0000e+00])
tensor([1.0000e+00, 1.0000e+00])
tensor([-1.0000e+00, -1.0000e+00])
tensor([1.0000e+00])
Fim época 0
tensor([-1.0000e+00, 1.0000e+00])
tensor([-1.0000e+00, 1.0000e+00])
tensor([1.0000e+00, 1.0000e+00])
tensor([-1.0000e+00])
Fim época 1
tensor([-1.0000e+00, -1.0000e+00])
tensor([-1.0000e+00, 1.0000e+00])
tensor([1.0000e+00, 1.0000e+00])
tensor([1.0000e+00])
Fim época 2
tensor([1.0000e+00, -1.0000e+00])
tensor([1.0000e+00, 1.0000e+00])
tensor([-1.0000e+00, -1.0000e+00])
tensor([1.0000e+00])
Fim época 3
tensor([1.0000e+00, -1.0000e+00])
tensor([-1.0000e+00, 1.0000e+00])
tensor([1.0000e+00, -1.0000e+00])
tensor([1.0000e+00])
Fim época 4
tensor([1.0000e+00, 1.0000e+00])
tensor([1.0000e+00, 1.0000e+00])
tensor([-1.0000e+00, -1.0000e+00])
tensor([-1.0000e+00])
Fim época 5
tensor([1.0000e+00, -1.0000e+00])
tensor([1.0000e+00, 1.0000e+00])
tensor([-1.0000e+00, 1.0000e+00])
tensor([-1.0000e+00])
Fim época 6
tensor([-1.0000e+00, 1.0000e+00])
tensor([-1.0000e+00, 

In [ ]:
del dummy_loader, dummy_dataset

# Treinamento do ranqueador

Fonte: https://colab.research.google.com/drive/10etP7Lb915EC-uEuf1IKC8DYkyg_om6-?usp=sharing

In this notebook, we will finetuned and evaluate a BERT-based model on a document classification task.

Our dataset will be a smaller version of the IMDB Sentiment Analysis dataset.

To finetune and evaluate faster, we will be using [miniLM](https://huggingface.co/microsoft/MiniLM-L12-H384-uncased), which is small BERT-like model distilled from a larger one.

## Carregando o Dataset e os Dataloaders


Independente da configuração do leiaute de treinamento, a validação é no leiaute qp

In [ ]:
datasets_carregados_previamente = False #@param {type:"boolean"}

print(datasets_carregados_previamente)

False


In [ ]:
import io

In [ ]:
%%time
if datasets_carregados_previamente:
  with open(PATH_TRAIN_LOCAL+'/dataset_valid_len_'+str(hparam['num_sentenca_valid'])+'.pt','rb') as f:
    buffer = io.BytesIO(f.read())
  dataset_valid = torch.load(buffer)
  with open(PATH_TRAIN_LOCAL+'/dataset_train'+'_len_'+str(hparam['num_sentenca_train'])+'.pt','rb') as f:
    buffer = io.BytesIO(f.read())
  dataset_train = torch.load(buffer)
  print('Datasets carregados')

CPU times: user 0 ns, sys: 5 µs, total: 5 µs
Wall time: 7.87 µs


In [ ]:
%%time
if not datasets_carregados_previamente:
  print("carregando valid_dataset")
  dataset_valid = MyDataset(texts=X_valid, classes=Y_valid, tokenizer=tokenizer, max_seq_length=hparam['max_seq_length'])
  torch.save(dataset_valid, PATH_TRAIN_LOCAL+'/dataset_valid_len_'+str(hparam['num_sentenca_valid'])+'.pt')

carregando valid_dataset


encoding text pair: 100%|██████████| 2827/2827 [00:10<00:00, 262.84it/s]


	Vou converter lista para tensor;  Momento: [2023-Jun-24 18:50:11]
CPU times: user 6.23 s, sys: 128 ms, total: 6.36 s
Wall time: 11.9 s


In [ ]:
%%time
if not datasets_carregados_previamente:
  print("carregando train_dataset")
  dataset_train = MyDataset(texts=X_train, classes=Y_train, tokenizer=tokenizer, max_seq_length=hparam['max_seq_length'])
  torch.save(dataset_train, PATH_TRAIN_LOCAL+'/dataset_train'+'_len_'+str(hparam['num_sentenca_train'])+'.pt')

carregando train_dataset


encoding text pair: 100%|██████████| 279809/279809 [09:17<00:00, 502.09it/s]


	Vou converter lista para tensor;  Momento: [2023-Jun-24 18:59:30]
CPU times: user 9min 1s, sys: 8.5 s, total: 9min 10s
Wall time: 12min 22s


## Finetuning

### Funções auxiliares de treinamento

In [ ]:
gera_tag_rastro_experiencia_treino(parm_aula='indir-minilm', hparam=hparam)


'indir-minilm|max_seq_length 512|batch_size 32'

In [ ]:
def treina_modelo (model, hparam, parm_dataloader_train, parm_dataloader_valid,
                   parm_intervalo_print:int=1,
                   parm_se_salva_modelo:bool=True, parm_se_gera_rastro:bool=True):
    global PATH_TRAIN_LOCAL

    if parm_se_gera_rastro:
        rastro_neptune = NeptuneRastroRun(hparam, parm_lista_tag= gera_tag_rastro_experiencia_treino(parm_aula='indir-minilm', hparam=hparam) )
    history = []
    n_examples = 0
    time_inicio_treino = time.time()



    try:
        best_metrica_valid = 0 # float('-inf')
        best_epoch = 0


        train_losses = []
        correct = 0
        train_samples = 0
        model.train()
        se_para_execucao = False
        ultimo_step_treinado = 0
        for epoch in tqdm(range(hparam['num_epochs']), desc='Epochs'):

            if n_examples >= hparam['max_examples'] or se_para_execucao:
                break


            for cnt_step, batch in enumerate(tqdm(parm_dataloader_train, mininterval=0.5, desc='Train', disable=False)):

                if n_examples >= hparam['max_examples'] or se_para_execucao:
                    break
                ultimo_step_treinado += 1
                hparam['optimizer'].zero_grad()
                # ipdb.set_trace(context=3)
                outputs = model(input_ids=batch['input_ids'].to(hparam['device']),
                                attention_mask=batch['attention_mask'].to(hparam['device']),
                                token_type_ids=batch['token_type_ids'].to(hparam['device']),
                                labels=batch['labels'].to(hparam['device']))
                loss = outputs.loss
                loss.backward()
                hparam['optimizer'].step()
                hparam['scheduler'].step()
                n_examples += len(batch['input_ids'])  # Increment of batch size
                train_losses.append(loss.cpu().item())
                train_samples += batch['labels'].size(0)

                preds = torch.sign(outputs.logits).squeeze()
                correct += (preds == batch['labels'].to(hparam['device'])).sum().cpu().item()
                # ipdb.set_trace(context=2)
                if ultimo_step_treinado % hparam['eval_every_steps']==0:
                    if isinstance(parm_dataloader_train,torch.utils.data.dataloader.DataLoader):
                        accuracy_train = correct / train_samples
                    elif isinstance(parm_dataloader_train,list):  # usado para overfit em um batch
                        accuracy_train = correct / train_samples
                        assert len(parm_dataloader_train) == 1, f"Calculo de accuracy para overfit em um batch precisa ser revisado"
                    else:
                        raise Exception(f'parm_dataloader_train has type unexpected {type(parm_dataloader_train)} ')
                    loss_train_mean = mean(train_losses)
                    train_losses = []
                    correct = 0
                    train_samples = 0
                    accuracy_valid, mean_loss_valid = evaluate(model=model, dataloader=parm_dataloader_valid, set_name='Valid')
                    model.train()
                    # ipdb.set_trace(context=2)
                    print(f'Epoch: {epoch + 1} Step: {ultimo_step_treinado} Training loss: {loss_train_mean:0.2f} accuracy: {accuracy_train:0.3f} \nEm validação: loss: {mean_loss_valid:0.3f}; accuracy: {accuracy_valid:0.3f}')
                    metrica_rastro = {"train/loss": loss_train_mean,
                                      "train/accuracy": accuracy_train,
                                      "train/steps": ultimo_step_treinado,
                                      "train/n_examples": n_examples,
                                      "train/learning_rate": hparam["optimizer"].param_groups[0]["lr"],
                                      "valid/loss": mean_loss_valid,
                                      "valid/accuracy": accuracy_valid}
                    history.append(metrica_rastro)
                    if parm_se_gera_rastro:
                        rastro_neptune.salva_metrica(metrica_rastro)

                    if parm_intervalo_print > 0:
                        if (ultimo_step_treinado)%(parm_intervalo_print*hparam['eval_every_steps']) == 0:
                            print(f'Step: {ultimo_step_treinado} Amostras:{n_examples:d}  {100*n_examples/hparam["max_examples"]:.3f}%  Momento: {time.strftime("[%Y-%b-%d %H:%M:%S]")}')
                            print(f'lr: {hparam["optimizer"].param_groups[0]["lr"]:.5e} Train loss: {loss_train_mean:.4f}  accuracy: {accuracy_train:.4f}')
                            print(f'Valid loss: {mean_loss_valid:.4f} accuracy: {accuracy_valid:.4f}  ')

                    if accuracy_valid > best_metrica_valid:
                        best_model_dict = model.state_dict()
                        best_metrica_valid = accuracy_valid
                        best_step = ultimo_step_treinado
                        if parm_se_salva_modelo:
                          path_modelo = f'{PATH_TRAIN_LOCAL}/model_epoch_{best_step}_treino_{time.strftime("%Y-%b-%d %H:%M:%S")}.pt'
                          # torch.save(model, path_modelo)
                          model.save_pretrained(path_modelo)

                        # print(f'encontrado best model Epoch: {epoch + 1} ')
                    elif hparam['early_stop'] < ( ultimo_step_treinado + 1 - best_step):
                        print(f"Parando por critério de early_stop na época {epoch + 1} step {ultimo_step_treinado} sendo best_step {best_step} e ealy_stop {hparam['early_stop']}")
                        se_para_execucao = True

        # fim do treino
        if isinstance(parm_dataloader_train,torch.utils.data.dataloader.DataLoader):
            accuracy_train = correct / len(parm_dataloader_train.dataset)
        elif isinstance(parm_dataloader_train,list):  # usado para overfit em um batch
            accuracy_train = correct / len(parm_dataloader_train[0]['labels'])
            assert len(parm_dataloader_train) == 1, f"Calculo de accuracy para overfit em um batch precisa ser revisado"

        loss_train_mean = mean(train_losses)
        accuracy_valid, mean_loss_valid = evaluate(model=model, dataloader=parm_dataloader_valid, set_name='Valid')

        print(f'Epoch: {epoch + 1} Step: {ultimo_step_treinado} Training loss: {loss_train_mean:0.2f} accuracy: {accuracy_train:0.3f} \nEm validação: loss: {mean_loss_valid:0.3f}; accuracy: {accuracy_valid:0.3f}')
        metrica_rastro = {"train/loss": loss_train_mean,
                          "train/accuracy": accuracy_train,
                          "train/steps": ultimo_step_treinado,
                          "train/n_examples": n_examples,
                          "train/learning_rate": hparam["optimizer"].param_groups[0]["lr"],
                          "valid/loss": mean_loss_valid,
                          "valid/accuracy": accuracy_valid}
        history.append(metrica_rastro)
        if parm_se_gera_rastro:
            rastro_neptune.salva_metrica(metrica_rastro)

        print(f'** END ** Step: {ultimo_step_treinado} Amostras:{n_examples:d}  {100*n_examples/hparam["max_examples"]:.3f}%  Momento: {time.strftime("[%Y-%b-%d %H:%M:%S]")}')
        print(f'lr: {hparam["optimizer"].param_groups[0]["lr"]:.5e} Train loss: {loss_train_mean:.4f}  accuracy: {accuracy_train:.4f}')
        print(f'Valid loss: {mean_loss_valid:.4f} accuracy: {accuracy_valid:.4f}  ')


        if parm_se_salva_modelo:
            path_modelo = f'{PATH_TRAIN_LOCAL}/model_fim_treino_{time.strftime("%Y-%b-%d %H:%M:%S")}.pt'
            # torch.save(model, path_modelo)
            model.save_pretrained(path_modelo)
            print(f"Modelo final  (step {ultimo_step_treinado}) salvo em {path_modelo}")

            if best_model_dict is not None:
                model.load_state_dict(best_model_dict)
                model.to(hparam['device'])
                path_modelo = f'{PATH_TRAIN_LOCAL}/model_fim_treino_{time.strftime("%Y-%b-%d %H:%M:%S")}.pt'
                # torch.save(model, path_modelo)
                model.save_pretrained(path_modelo)
                print(f"Modelo com melhor resultado em validação (step {best_step}) salvo após treino em {path_modelo}")

        # calculando tempo gasto e médio por
        tempo_treino = time.time() - time_inicio_treino
        if parm_se_gera_rastro:
            rastro_neptune.run_neptune["context/tempo_treino"] = tempo_treino
            rastro_neptune.run_neptune["context/best_epoch"] = best_epoch
            rastro_neptune.run_neptune["context/tempo_treino_epoc_mean"] = tempo_treino/hparam['num_epochs']
            # rastro_neptune.gera_grafico_modelo(parm_dataloader_train, model)
    finally: # para não deixar em aberto a execução no Neptune
        if parm_se_gera_rastro:
            rastro_neptune.stop()
    return history

In [ ]:
def count_parameters(model):
    """
    Conta o número de parâmetros treináveis em um modelo PyTorch.

    Args:
        model (torch.nn.Module): O modelo PyTorch a ser avaliado.

    Returns:
        int: O número de parâmetros treináveis do modelo.

    """
    # Retorna a soma do número de elementos em cada tensor de parâmetro que requer gradiente.
    # A propriedade "requires_grad" é definida como True para todos os tensores de parâmetro que
    # precisam ser treinados, enquanto que para aqueles que não precisam, ela é definida como False.
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
from statistics import mean, stdev

In [ ]:
# We first define the evaluation function to measure accuracy and loss
def evaluate(model, dataloader, set_name):
    losses = []
    correct = 0
    model.eval()
    with torch.no_grad():
        for ndx, batch in tqdm(enumerate(dataloader), mininterval=0.5, desc=set_name, disable=False, total=len(dataloader)):
            input_ids = batch['input_ids'].to(hparam['device'])
            token_type_ids = batch['token_type_ids'].to(hparam['device'])
            attention_mask = batch['attention_mask'].to(hparam['device'])
            labels = batch['labels'].to(hparam['device'])
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, labels=labels)
            loss_val = outputs.loss
            losses.append(loss_val.cpu().item())

            preds = torch.sign(outputs.logits).squeeze()
            correct += (preds == labels).sum().item()

            # if ndx < 5:
            #   print(f'Em evaluete. Batch {ndx}')
            #   print(f'loss_val: {loss_val} ')
            #   print(f'preds: {preds} \nlabels: {labels} ')
            #   print(f'correct: {correct}')

    # ipdb.set_trace(context=3)
    if isinstance(dataloader,torch.utils.data.dataloader.DataLoader):
        accuracy = correct / len(dataloader.dataset)
    elif isinstance(dataloader,list):  # usado para overfit em um batch
        accuracy = correct / len(dataloader[0]['labels'])
        assert len(dataloader) == 1, f"Calculo de accuracy para overfit em um batch precisa ser revisado"
    mean_loss = mean(losses)
    return accuracy, mean_loss

In [ ]:
from torch import nn
from torch import optim
from tqdm.auto import tqdm
from transformers import get_linear_schedule_with_warmup

In [ ]:
def inicializa_modelo():
  global model, hparam, dataloader_train

  model = AutoModelForSequenceClassification.from_pretrained(hparam['model_name']).to(hparam['device'])
  hparam['learning_rate']= 1e-6 # 3e-5 # 1e-3
  hparam['num_params'] = count_parameters(model)
  print(f"Number of model parameters: {hparam['num_params']}")
  hparam['amsgrad']= False
  hparam['num_epochs'] = 4
  hparam['weight_decay'] = 1e-4
  hparam['early_stop'] = 2000
  hparam['batch_size'] = 32
  hparam['num_step_epoch'] = len(dataloader_train)
  hparam['num_training_steps'] = hparam['num_epochs'] * hparam['num_step_epoch']
  hparam['max_examples'] = hparam['num_training_steps'] * hparam['batch_size']
  hparam['eval_every_steps'] = 500
  hparam['num_warmup_steps'] = 500 # int(hparam['num_training_steps']  * 0.05)
  hparam['optimizer'] = torch.optim.Adam(model.parameters(), lr=hparam['learning_rate'], weight_decay= hparam['weight_decay'], amsgrad=hparam['amsgrad'])
  hparam['scheduler'] = get_linear_schedule_with_warmup(hparam['optimizer'], hparam['num_warmup_steps'], hparam['num_training_steps'])
  hparam['step_eval'] = 1000


In [ ]:
dataloader_train = DataLoader(dataset_train, batch_size=hparam['batch_size'], shuffle=True)
dataloader_valid = DataLoader(dataset_valid, batch_size=hparam['batch_size'], shuffle=False)

In [ ]:
inicializa_modelo()

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--unicamp-dl--mMiniLM-L6-v2-pt-v2/snapshots/a9c0d68bae51b053e85747605350bb781d945712/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "unicamp-dl/mMiniLM-L6-v2-pt-v2",
  "architectures": [
    "XLMRobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 384,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "sbert_ce_default_activation_function": "torch.nn.modules.linear.Identity",
  "torch_dtype": "float32",
  "transformers_version"

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--unicamp-dl--mMiniLM-L6-v2-pt-v2/snapshots/a9c0d68bae51b053e85747605350bb781d945712/pytorch_model.bin
All model checkpoint weights were used when initializing XLMRobertaForSequenceClassification.

All the weights of XLMRobertaForSequenceClassification were initialized from the model checkpoint at unicamp-dl/mMiniLM-L6-v2-pt-v2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use XLMRobertaForSequenceClassification for predictions without further training.


Number of model parameters: 106994305


Depois tentar definir:hparam[ hparam['dim_feedforward']=256

In [ ]:
raise Exception('parar aqui')

Exception: ignored

### Evaluating on the valid dataset before training

In [ ]:
%%time
accuracy, mean_loss = evaluate(model=model, dataloader=dataloader_valid, set_name='Valid')
print(f'Em validação: loss: {mean_loss:0.3f}; accuracy: {accuracy:0.3f}')

### Testando em poucos dados: um batch apenas. Esperado overfit loss próxima de zero.

Por algum motivo a investigar quando passo dataset como lista não está gerando aprendizado. Mas quando passo normal, o modelo está aprendendo.




In [ ]:
hparam['num_epochs'] = 5
hparam['early_stop'] = 3
hparam['eval_every_steps'] = 1

In [ ]:
history =  treina_modelo (model, hparam,  [next(iter(dataloader_train))], [next(iter(dataloader_valid))], parm_se_salva_modelo=True, parm_se_gera_rastro=True)

Treinando em todos os dados

In [ ]:
ultimo_step_treinado , correct, cnt_step, n_examples, train_losses
accuracy_train, accuracy_valid, mean_loss_valid



In [ ]:
torch.cuda.empty_cache()

In [ ]:
inicializa_modelo()

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--unicamp-dl--mMiniLM-L6-v2-pt-v2/snapshots/a9c0d68bae51b053e85747605350bb781d945712/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "unicamp-dl/mMiniLM-L6-v2-pt-v2",
  "architectures": [
    "XLMRobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 384,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "sbert_ce_default_activation_function": "torch.nn.modules.linear.Identity",
  "torch_dtype": "float32",
  "transformers_version"

Number of model parameters: 106994305


In [ ]:
mostra_memoria(['cpu', 'gpu'])

Your runtime RAM in gb: 
 total 13.61
 available 1.79
 used 11.48
 free 1.02
 cached 1.08
 buffers 0.03
/nGPU
Sat Jun 24 20:07:39 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P0    28W /  70W |   5085MiB / 15360MiB |     18%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+--

### Treino do conjunto completo

In [ ]:
hparam

{'num_workers_dataloader': 0,
 'device': device(type='cuda', index=0),
 'percent_test_size': 0.01,
 'model_name': 'unicamp-dl/mMiniLM-L6-v2-pt-v2',
 'vocab_size': 250002,
 'batch_size': 32,
 'num_epochs': 4,
 'max_seq_length': 512,
 'num_sentenca_train': 279809,
 'num_sentenca_valid': 2827,
 'learning_rate': 1e-06,
 'num_params': 106994305,
 'amsgrad': False,
 'weight_decay': 0.0001,
 'early_stop': 2000,
 'num_step_epoch': 8745,
 'num_training_steps': 34980,
 'max_examples': 1119360,
 'eval_every_steps': 500,
 'num_warmup_steps': 500,
 'optimizer': Adam (
 Parameter Group 0
     amsgrad: False
     betas: (0.9, 0.999)
     capturable: False
     differentiable: False
     eps: 1e-08
     foreach: None
     fused: None
     initial_lr: 1e-06
     lr: 0.0
     maximize: False
     weight_decay: 0.0001
 ),
 'scheduler': <torch.optim.lr_scheduler.LambdaLR at 0x7ff2bb66f820>,
 'step_eval': 1000}

In [ ]:
history = treina_modelo (model, hparam,  dataloader_train, dataloader_valid, parm_intervalo_print=1)

https://app.neptune.ai/marcusborela/IA386DD/e/IAD-101


Epochs:   0%|          | 0/4 [00:00<?, ?it/s]

Train:   0%|          | 0/8745 [00:00<?, ?it/s]

Valid:   0%|          | 0/89 [00:00<?, ?it/s]

Configuration saved in /content/drive/MyDrive/treinamento/202301_IA368DD/indir/train/minilm/model_epoch_500_treino_2023-Jun-24 20:17:06.pt/config.json


...

Valid:   0%|          | 0/89 [00:00<?, ?it/s]

Epoch: 3 Step: 20500 Training loss: 0.54 accuracy: 0.807 
Em validação: loss: 0.532; accuracy: 0.817
Step: 20500 Amostras:655938  1875.180%  Momento: [2023-Jun-25 02:09:34]
lr: 4.19954e-07 Train loss: 0.5400  accuracy: 0.8073
Valid loss: 0.5319 accuracy: 0.8171  


Valid:   0%|          | 0/89 [00:00<?, ?it/s]

Epoch: 3 Step: 21000 Training loss: 0.54 accuracy: 0.810 
Em validação: loss: 0.529; accuracy: 0.818
Step: 21000 Amostras:671938  1920.921%  Momento: [2023-Jun-25 02:17:55]
lr: 4.05452e-07 Train loss: 0.5427  accuracy: 0.8099
Valid loss: 0.5286 accuracy: 0.8182  
Parando por critério de early_stop na época 3 step 21000 sendo best_step 19000 e ealy_stop 2000
Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 7 operations to synchronize with Neptune. Do not kill this process.
All 7 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/marcusborela/IA386DD/e/IAD-101/metadata


StatisticsError: ignored

Continuando o treinamento do modelo interrompida por erro de código (rastro: IAD-10, IAD-11).

In [ ]:
print(history)